In [3]:
import pandas as pd
import numpy as np

In [42]:
ADNI3_USB2="niis/ADNI3/USB2/patient_mri_mapping.csv"
ADNI3_USB1="niis/ADNI3/USB1/patient_mri_mapping.csv"
ADNI4="niis/ADNI4/patient_mri_mapping.csv"
PATIENTS="patient_information.csv"

data1=pd.read_csv(ADNI4)
data2=pd.read_csv(PATIENTS)


In [43]:
# VISDATE가 날짜 형식인지 확인하는 함수
def is_date_format(date):
    try:
        pd.to_datetime(date, format="%Y-%m-%d")  # 기본 형식 가정
        return True
    except ValueError:
        return False

# VISDATE 날짜 형식 확인
data1["Valid_VISDATE"] = data1["VISDATE"].apply(is_date_format)

# 조건 1: VISDATE가 유효하지 않은 경우 PTID 기준 join
data_invalid_visdate = data1[~data1["Valid_VISDATE"]]
result_invalid = pd.merge(
    data_invalid_visdate.drop(columns=["VISDATE"]),
    data2,
    how="left",
    on="PTID"
)
result_invalid["MRIDATE"] = None  # VISDATE가 날짜 형식이 아니므로 MRIDATE는 공란

# 조건 2: VISDATE가 유효한 경우 PTID와 VISDATE==MRI 기준 join
data_valid_visdate = data1[data1["Valid_VISDATE"]]
result_valid = pd.merge(
    data_valid_visdate,
    data2,
    how="left",
    left_on=["PTID", "VISDATE"],
    right_on=["PTID", "MRI"]
)
result_valid["MRIDATE"] = result_valid["MRI"]  # MRIDATE에 MRI 값 설정

# 조건별 결과 합치기
final_result = pd.concat([result_invalid, result_valid], ignore_index=True)

# 열 정리
final_result = final_result[["PHASE", "PTID", "MRI Number", "MRIDATE", "생년월일", "education", "MMSCORE"]]
final_result.rename(columns={"MRINumber": "MRIID"}, inplace=True)

# education과 MMSCORE를 int로 변환
final_result["education"] = pd.to_numeric(final_result["education"], errors="coerce").fillna(0).astype(int)
final_result["MMSCORE"] = pd.to_numeric(final_result["MMSCORE"], errors="coerce").fillna(0).astype(int)

# 결과 저장
final_result.to_csv("merged_output.csv", index=False)

print("결합된 데이터가 'merged_output.csv'에 저장되었습니다.")

결합된 데이터가 'merged_output.csv'에 저장되었습니다.


## ADNI3 metadata concat

In [44]:
data1=pd.read_csv("metadatas/ADNI3_USB.csv")
data2=pd.read_csv("metadatas/merged_output_ADNI3_USB2.csv")
merged_data = pd.concat([data1, data2], ignore_index=True)

sorted_data = merged_data.sort_values(by="PTID")

sorted_data.to_csv("metadatas/ADNI3_metadata.csv", index=False)

print("PTID 기준으로 정렬된 데이터가 'metadatas/ADNI3_metadata.csv'에 저장되었습니다.")


PTID 기준으로 정렬된 데이터가 'metadatas/ADNI3_metadata.csv'에 저장되었습니다.


## 매핑 안되는 데이터들 처리

In [ ]:
data1=pd.read_csv("metadatas/ADNI3_metadata.csv")
data2=pd.read_csv("metadatas/ADNI4_metadata.csv")

data1_cleaned=data1.dropna(subset=["PHASE"])
data2_cleaned=data2.dropna(subset=["PHASE"])

data1_cleaned.to_csv('metadatas/ADNI3_metadata_cleaned.csv',index=False)
data2_cleaned.to_csv('metadatas/ADNI4_metadata_cleaned.csv',index=False)